API_key = '26DTN6Q1KHV06GLOCLVXX5YO'

In [5]:
import pandas as pd

# Load CSV correctly
df_sites = pd.read_csv(r"D:\clearsky\Chelsea sensor list.csv")

# Confirm structure
print(df_sites.columns.tolist())

# Split GPS Coordinates into latitude & longitude
df_sites[["Latitude", "Longitude"]] = (
    df_sites["GPS Coordinates"]
    .str.split(",", expand=True)
)

# Convert to float
df_sites["Latitude"] = df_sites["Latitude"].astype(float)
df_sites["Longitude"] = df_sites["Longitude"].astype(float)

# Drop original GPS column (optional)
df_sites = df_sites.drop(columns=["GPS Coordinates"])

print(df_sites.head())

['ID', 'City', 'Location Description', 'GPS Coordinates']
          ID     City                               Location Description  \
0  MOD-00142  Chelsea                iSUPER 142 @ Broadway & Mills Creek   
1  MOD-00143  Chelsea             iSUPER 143 @ Eastern Ave & Griffin Way   
2  MOD-00146  Chelsea                       iSUPER 146 @ 228 Everett Ave   
3  MOD-00147  Chelsea                       iSUPER 147 @ Washington Park   
4  MOD-00149  Chelsea  iSUPER 149 @ Mary O'Malley State Park pier (li...   

   Latitude  Longitude  
0  42.40279  -71.01839  
1  42.39082  -71.02260  
2  42.39809  -71.04047  
3  42.40555  -71.03589  
4  42.38752  -71.04957  


In [9]:
import requests, pandas as pd

auth = ("SDGBROHD6RGPCJUFX97WFPZG", "")

r = requests.get("https://api.quant-aq.com/v1/devices", auth=auth)
print(r.status_code)
print(r.text[:3000])  # don't print huge

200
{"data":[],"meta":{"first_url":"https://api.quant-aq.com/v1/devices/?page=1&per_page=50","last_url":"https://api.quant-aq.com/v1/devices/?page=0&per_page=50","next_url":null,"page":1,"pages":0,"per_page":50,"prev_url":null,"total":0}}



In [14]:
import requests, pandas as pd

AUTH = ("JFQNWVEWQUFTM9T46NFNZSF8", "")

r = requests.get("https://api.quant-aq.com/v1/devices", auth=AUTH)
print("Devices endpoint:", r.status_code)
print(r.text[:2000])  # quick view

Devices endpoint: 200
{"data":[{"city":"","country":"US","created":"2022-11-10T23:50:22.824945","description":"iSUPER 141 ","geo":{"lat":null,"lon":null},"id":1397,"last_seen":"2025-09-29T16:29:45","model":"modulair","n_datapoints":null,"outdoors":true,"owner_id":null,"private":true,"sn":"MOD-00141","status":"ACTIVE","timezone":"US/Eastern","url":"https://api.quant-aq.com/v1/devices/MOD-00141"},{"city":"Chelsea","country":"US","created":"2022-11-11T00:35:50.338546","description":"iSUPER 142 @ Broadway & Mills Creek - TND","geo":{"lat":42.4029,"lon":-71.0185},"id":1398,"last_seen":"2026-02-20T12:41:48","model":"modulair","n_datapoints":null,"outdoors":true,"owner_id":null,"private":false,"sn":"MOD-00142","status":"ACTIVE","timezone":"US/Eastern","url":"https://api.quant-aq.com/v1/devices/MOD-00142"},{"city":"Chelsea","country":"US","created":"2022-11-11T00:44:07.819930","description":"iSUPER 143 @ Eastern Ave & Griffin Way","geo":{"lat":42.3909,"lon":-71.0228},"id":1399,"last_seen":"202

In [1]:
import requests
import pandas as pd

AUTH = ("JFQNWVEWQUFTM9T46NFNZSF8", "")

# Load sites
sites = pd.read_csv(r"D:\clearsky\Chelsea sensor list.csv")
sites["ID"] = sites["ID"].astype(str).str.strip()

# Pull devices
r = requests.get("https://api.quant-aq.com/v1/devices?per_page=200", auth=AUTH)
print("Devices endpoint:", r.status_code)
r.raise_for_status()

devices = pd.DataFrame(r.json().get("data", []))
print("Devices returned:", len(devices))
print("Device columns:", devices.columns.tolist())

# ✅ Key point: match CSV IDs to devices['sn']
if "sn" not in devices.columns:
    print("❌ devices has no 'sn' column. Show first rows:")
    print(devices.head())
else:
    matches = sites["ID"].isin(devices["sn"].astype(str)).sum()
    print("Matches using devices['sn']:", matches, "out of", len(sites))

    # Show a few examples to confirm mapping visually
    merged = sites.merge(
        devices[["sn", "id", "description", "last_seen", "status"]],
        left_on="ID", right_on="sn", how="left"
    )
    print("Unmatched rows:", merged["id"].isna().sum())
    display(merged[["ID", "description", "last_seen", "status"]].head(15))

    # Pull data (limit 100)
    all_sensor_data = []
    for sn in merged["sn"].dropna().astype(str).unique():
        url = f"https://api.quant-aq.com/v1/devices/{sn}/data?limit=100"
        resp = requests.get(url, auth=AUTH)

        if resp.status_code == 200:
            data_list = resp.json().get("data", [])
            if data_list:
                temp = pd.DataFrame(data_list)
                temp["sn"] = sn
                all_sensor_data.append(temp)
                print(f"✅ {sn}: {len(temp)} rows")
            else:
                print(f"⚠️ {sn}: no data returned")
        else:
            print(f"❌ {sn}: {resp.status_code} {resp.text[:200]}")

    if all_sensor_data:
        final = pd.concat(all_sensor_data, ignore_index=True)
        out_path = r"D:\clearsky\sensor_data.csv"
        final.to_csv(out_path, index=False)
        print("\nSaved:", out_path)
        print("Columns:", final.columns.tolist())
        display(final.head())
    else:
        print("\nNo sensor data pulled (even though mapping worked).")

Devices endpoint: 200
Devices returned: 180
Device columns: ['city', 'country', 'created', 'description', 'geo', 'id', 'last_seen', 'model', 'n_datapoints', 'outdoors', 'owner_id', 'private', 'sn', 'status', 'timezone', 'url']
Matches using devices['sn']: 85 out of 85
Unmatched rows: 0


,ID,description,last_seen,status
0,MOD-00142,iSUPER 142 @ Broadway & Mills Creek - TND,2026-02-20T12:41:48,ACTIVE
1,MOD-00143,iSUPER 143 @ Eastern Ave & Griffin Way,2026-02-22T04:14:52,ACTIVE
2,MOD-00146,iSUPER 146 @ 228 Everett Ave,2026-02-22T04:14:32,ACTIVE
3,MOD-00147,iSUPER 147 @ Washington Park,2026-02-21T06:12:30,ACTIVE
4,MOD-00149,iSUPER 149 @ Mary O'Malley State Park pier (li...,2026-02-22T04:13:23,ACTIVE
5,MOD-00150,iSUPER 150 @ Bassett Square,2026-02-21T12:47:57,ACTIVE
6,MOD-00151,iSUPER 151 @ Marginal & Charles St,2026-02-22T04:13:12,ACTIVE
7,MOD-00152,iSUPER 152 @ Island End Park Boardwalk,2026-02-22T04:12:55,ACTIVE
8,MOD-00155,iSUPER 155 @ Scrivano Apartments Basketball Co...,2026-02-20T21:52:49,ACTIVE
9,MOD-00156,iSUPER 156 @ Buckley Apartments - CHA,2026-02-21T01:04:27,ACTIVE


✅ MOD-00142: 50 rows
✅ MOD-00143: 50 rows
✅ MOD-00146: 50 rows
✅ MOD-00147: 50 rows
✅ MOD-00149: 50 rows
✅ MOD-00150: 50 rows
✅ MOD-00151: 50 rows
✅ MOD-00152: 50 rows
✅ MOD-00155: 50 rows
✅ MOD-00156: 50 rows
✅ MOD-00157: 50 rows
✅ MOD-00158: 50 rows
✅ MOD-00159: 50 rows
✅ MOD-00160: 50 rows
✅ MOD-00161: 50 rows
✅ MOD-00162: 50 rows
✅ MOD-00163: 50 rows
✅ MOD-00164: 50 rows
✅ MOD-00165: 50 rows
✅ MOD-00166: 50 rows
✅ MOD-00169: 50 rows
✅ MOD-00170: 50 rows
✅ MOD-00172: 50 rows
✅ MOD-00173: 50 rows
✅ MOD-00175: 50 rows
✅ MOD-00176: 50 rows
✅ MOD-00177: 50 rows
✅ MOD-00178: 50 rows
✅ MOD-00180: 50 rows
✅ MOD-00182: 50 rows
✅ MOD-00183: 50 rows
✅ MOD-00186: 50 rows
✅ MOD-00188: 50 rows
✅ MOD-00189: 50 rows
✅ MOD-00190: 50 rows
✅ MOD-00191: 50 rows
✅ MOD-00195: 50 rows
✅ MOD-00196: 50 rows
✅ MOD-00197: 50 rows
✅ MOD-00200: 50 rows
✅ MOD-00203: 50 rows
✅ MOD-00206: 50 rows
✅ MOD-00213: 50 rows
✅ MOD-00214: 50 rows
✅ MOD-00216: 50 rows
✅ MOD-00223: 50 rows
✅ MOD-00226: 50 rows
✅ MOD-00227: 

C:\Users\hp\AppData\Local\Temp\ipykernel_43980\1237344507.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat(all_sensor_data, ignore_index=True)


,co,co2,geo,met,model,no,no2,o3,pm1,pm10,...,raw_data_id,rh,sn,temp,timestamp,timestamp_local,url,wd,ws,ws_scalar
0,688.95,508.19,"{'lat': 42.4029, 'lon': -71.0185}","{'rh': 71.1, 'temp': 1.7, 'wd': 119.6, 'ws': 0...","{'gas': {'co': 19520, 'co2': 15558, 'no': 1719...",58.00,5.74,14.31,4.17,5.28,...,614343752,71.1,MOD-00142,1.7,2026-02-20T12:41:48,2026-02-20T07:41:48,https://api.quant-aq.com/v1/devices/MOD-00142/...,NaN,NaN,NaN
1,682.39,507.94,"{'lat': 42.4029, 'lon': -71.0185}","{'rh': 71.1, 'temp': 1.7, 'wd': 127.3, 'ws': 0...","{'gas': {'co': 19520, 'co2': 15558, 'no': 1719...",58.78,6.01,14.64,4.20,5.80,...,614343754,71.1,MOD-00142,1.7,2026-02-20T12:40:48,2026-02-20T07:40:48,https://api.quant-aq.com/v1/devices/MOD-00142/...,NaN,NaN,NaN
2,669.27,508.43,"{'lat': 42.4029, 'lon': -71.0185}","{'rh': 71.1, 'temp': 1.7, 'wd': 134.3, 'ws': 0...","{'gas': {'co': 19520, 'co2': 15558, 'no': 1719...",56.96,6.29,14.06,4.20,15.80,...,614343750,71.1,MOD-00142,1.7,2026-02-20T12:39:48,2026-02-20T07:39:48,https://api.quant-aq.com/v1/devices/MOD-00142/...,NaN,NaN,NaN
3,679.34,512.64,"{'lat': 42.4029, 'lon': -71.0185}","{'rh': 71.2, 'temp': 1.6, 'wd': 135.7, 'ws': 0...","{'gas': {'co': 19520, 'co2': 15558, 'no': 1719...",57.84,6.31,13.61,4.81,10.03,...,614343751,71.2,MOD-00142,1.6,2026-02-20T12:38:48,2026-02-20T07:38:48,https://api.quant-aq.com/v1/devices/MOD-00142/...,NaN,NaN,NaN
4,714.61,513.24,"{'lat': 42.4029, 'lon': -71.0185}","{'rh': 71.2, 'temp': 1.6, 'wd': 88.0, 'ws': 0....","{'gas': {'co': 19520, 'co2': 15558, 'no': 1719...",58.10,6.62,14.40,4.43,15.73,...,614341524,71.2,MOD-00142,1.6,2026-02-20T12:37:48,2026-02-20T07:37:48,https://api.quant-aq.com/v1/devices/MOD-00142/...,NaN,NaN,NaN


In [3]:
print(final.columns.tolist())

['co', 'co2', 'geo', 'met', 'model', 'no', 'no2', 'o3', 'pm1', 'pm10', 'pm25', 'raw_data_id', 'rh', 'sn', 'temp', 'timestamp', 'timestamp_local', 'url', 'wd', 'ws', 'ws_scalar']
